In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import trend_finder_lr as tflr
import trend_finder_ovr as tfovr
import ml_models as ml
import trend_finder_ovr as ovr
import yfinance as yf
import pickle
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, svm
from imblearn.over_sampling import RandomOverSampler

# Run a test looking for stock price increases and decreases of percent, 30 days out
- Record 60 trading days before the trend
- use those 60 trading days as features

In [2]:
tesla = tflr.Historydf('TSLA', '1y', 30, 5, 60)

In [3]:
tesla.df

,Open,Close,Volume
0,291.913330,289.913330,55843200
1,291.453339,296.453339,63984900
2,297.563324,297.096680,57259800
3,302.359985,296.070007,53230000
4,297.429993,288.089996,57163900
...,...,...,...
246,238.729996,232.960007,88197600
247,228.020004,225.600006,112484500
248,226.059998,219.220001,120718400
249,214.119995,215.490005,135813700


# Find trends, enter [Buy/Sell] classifier, and create new dataframe

In [4]:
tesla.find_trends()

start index = 59, end index = 221


In [5]:
tesla.pretrends_df

,Open,Close,Volume,Buy/Sell
0,221.551193,231.279999,133098020.0,NaN
1,291.913330,289.913330,55843200.0,NaN
2,291.453339,296.453339,63984900.0,NaN
3,297.563324,297.096680,57259800.0,NaN
4,302.359985,296.070007,53230000.0,NaN
...,...,...,...,...
7675,260.600006,261.769989,112267600.0,NaN
7676,276.489990,279.820007,119685900.0,NaN
7677,278.820007,282.480011,131530900.0,NaN
7678,278.089996,276.540009,120332100.0,NaN


# Structure the data in preperation for training model and normalize the values

In [6]:
tesla.structure_data()

In [7]:
tesla.structured_df

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,Buy/Sell
0,0.334357,0.869511,0.866013,0.912483,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,...,0.228251,0.232433,0.160841,0.622114,0.570535,0.952927,0.933847,0.995670,0.795092,0
1,0.869511,0.866013,0.912483,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,...,0.233443,0.161541,0.624819,0.573016,0.957071,0.937908,1.000000,0.798550,0.554413,0
2,0.866013,0.912483,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,...,0.161541,0.624819,0.573016,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0
3,0.912483,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,0.726422,...,0.624819,0.573016,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0
4,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,0.726422,0.788789,...,0.573016,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0.247377,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.309250,0.248510,0.222875,0.277982,0.311046,0.247530,0.255613,0.274961,0.281737,0.216018,...,0.623992,1.000000,0.634288,0.713324,0.741057,0.618757,0.576444,0.344522,0.189711,0
124,0.239346,0.214656,0.267731,0.299575,0.238402,0.246186,0.264822,0.271348,0.208052,0.106385,...,1.000000,0.634288,0.713324,0.741057,0.618757,0.576444,0.344522,0.189711,0.250105,0
125,0.210259,0.262246,0.293438,0.233518,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,...,0.634288,0.713324,0.741057,0.618757,0.576444,0.344522,0.189711,0.250105,0.346537,0
126,0.262246,0.293438,0.233518,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,...,0.713324,0.741057,0.618757,0.576444,0.344522,0.189711,0.250105,0.346537,0.255366,0


# Run logistic regression

In [8]:
tesla_lr = ml.logistic_regression('Tesla')

In [9]:
tesla_lr.train_model(tesla.structured_df)

The testing accuracy is: 1.00
Coefficients:
 [[ 0.15413368  0.01540626 -0.00983303  0.10536408  0.09291282  0.04371857
   0.18213561  0.12792762  0.02619427  0.12667552  0.11352169  0.12192781
   0.09845256  0.05153175 -0.07143201 -0.0621823  -0.08125078 -0.11164653
  -0.19325506 -0.058865   -0.01681912  0.0387854   0.15722449  0.15625223
   0.32024878  0.30108146  0.21451789  0.19184677  0.17795338  0.1648952
   0.19792226  0.18372872  0.10713125 -0.03097404  0.0157355  -0.02654765
  -0.00956522  0.04755218  0.02122526 -0.1142682  -0.07811982 -0.1781434
  -0.10669337 -0.04001657 -0.12434136 -0.17661833 -0.29003455 -0.31532023
  -0.24632499 -0.16522541 -0.1836552  -0.12961627 -0.18981555 -0.14059378
  -0.22271005 -0.10706708 -0.05915131  0.01137277  0.08970721  0.14101471
   0.27677754  0.00909206  0.03233585  0.11076178  0.03017682  0.09734578
   0.10752062  0.08532922  0.0195837   0.0614061   0.04575919  0.00123136
   0.01355569 -0.04329649 -0.13464166 -0.0860188  -0.09384122 -0.1869

# Adding a third classification for hold
- hold is represented by 2
- will include any points without increases or decreases of desired percentage
- model being used is One VS All

In [35]:
tesla = ovr.Historydf('TSLA', '5y', 30, 5, 60)

In [36]:
tesla.find_trends()

start index = 59, end index = 1227


In [37]:
tesla.pretrends_df

,Open,Close,Volume,Buy/Sell
0,221.551193,231.279999,133098020.0,NaN
1,21.391333,21.442667,89190000.0,NaN
2,21.275999,21.340000,77209500.0,NaN
3,21.379999,21.521334,54039000.0,NaN
4,21.200001,21.284666,196189500.0,NaN
...,...,...,...,...
70075,260.600006,261.769989,112267600.0,NaN
70076,276.489990,279.820007,119685900.0,NaN
70077,278.820007,282.480011,131530900.0,NaN
70078,278.089996,276.540009,120332100.0,NaN


In [38]:
tesla.structure_data()

In [39]:
tesla.structured_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,Buy/Sell
0,0.954660,0.021845,0.021307,0.021792,0.020953,0.021080,0.018551,0.016063,0.015982,0.014410,...,0.146354,0.139960,0.140726,0.105178,0.125533,0.116088,0.049795,0.111122,0.061437,2
1,0.697243,0.680087,0.695557,0.668782,0.672848,0.592126,0.512693,0.510115,0.459936,0.342027,...,0.139960,0.140726,0.105178,0.125533,0.116088,0.049795,0.111122,0.061437,0.047848,0
2,0.680087,0.695557,0.668782,0.672848,0.592126,0.512693,0.510115,0.459936,0.342027,0.339548,...,0.140726,0.105178,0.125533,0.116088,0.049795,0.111122,0.061437,0.047848,0.034050,0
3,0.676048,0.650024,0.653976,0.575518,0.498313,0.495807,0.447036,0.332434,0.330024,0.091952,...,0.105178,0.125533,0.116088,0.049795,0.111122,0.061437,0.047848,0.034050,0.119932,0
4,0.637550,0.641425,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,...,0.099177,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,2
5,0.641425,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,...,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0.120783,2
6,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,...,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0.120783,0.007032,2
7,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,...,0.093011,0.042313,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,2
8,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,...,0.042313,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,0.128688,2
9,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,...,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,0.128688,0.073206,2


In [40]:
from sklearn.multiclass import OneVsRestClassifier

In [41]:
tesla_lr = ml.one_vs_rest('Tesla')

In [56]:
for i in range(0, 200):
    tesla_lr.train_model(tesla.structured_df)
    if tesla_lr.accuracy >= 55:
        tesla_lr.save_current_model()

The testing accuracy is: 0.45
The testing accuracy is: 0.46
The testing accuracy is: 0.48
The testing accuracy is: 0.46
The testing accuracy is: 0.46
The testing accuracy is: 0.45
The testing accuracy is: 0.47
The testing accuracy is: 0.46
The testing accuracy is: 0.48
The testing accuracy is: 0.48
The testing accuracy is: 0.48
The testing accuracy is: 0.47
The testing accuracy is: 0.53
The testing accuracy is: 0.49
The testing accuracy is: 0.47
The testing accuracy is: 0.47
The testing accuracy is: 0.47
The testing accuracy is: 0.47
The testing accuracy is: 0.49
The testing accuracy is: 0.43
The testing accuracy is: 0.51
The testing accuracy is: 0.50
The testing accuracy is: 0.46
The testing accuracy is: 0.49
The testing accuracy is: 0.48
The testing accuracy is: 0.47
The testing accuracy is: 0.45
The testing accuracy is: 0.44
The testing accuracy is: 0.46
The testing accuracy is: 0.49
The testing accuracy is: 0.49
The testing accuracy is: 0.51
The testing accuracy is: 0.49
The testin